In [98]:
import numpy as np
import pandas as pd
#import pgeocode
import matplotlib.pyplot as plt
from pylab import rcParams
#import seaborn as sb

import networkx as nx
#from functools import reduce
from mlxtend.frequent_patterns import apriori,association_rules
from mlxtend.preprocessing import TransactionEncoder
import seaborn as sns
import collections
from nltk import flatten

In [131]:
df=pd.read_csv('converted_dataset.csv')
tff=pd.read_csv("training_first_fourth_city.csv")

In [173]:
tff

,first_city,fourth_city,count
0,8183,30628,1
1,38677,38677,178
2,64876,36063,208
3,55990,36063,1
4,11306,65690,1
...,...,...,...
104223,11349,41168,1
104224,64876,1766,1
104225,4476,9608,1
104226,51291,24036,1


In [132]:
first_city=33408

In [142]:
tff

,first_city,fourth_city,count
0,8183,30628,1
1,38677,38677,178
2,64876,36063,208
3,55990,36063,1
4,11306,65690,1
...,...,...,...
104223,11349,41168,1
104224,64876,1766,1
104225,4476,9608,1
104226,51291,24036,1


In [118]:
def lift_list(df,first_city,fourth_cities):
    
    dep_city_code=first_city
    arr_city_code=fourth_cities
    
    
    df_denominator=df['count'].sum()
    
    depart_city=df['first_city'].loc[lambda x: x==dep_city_code].index 
    count_city=0

    for i in range(len(depart_city)):
        #print(depart_city[i])
        if df['fourth_city'][depart_city[i]]==arr_city_code:
            count_city+=df['count'][depart_city[i]]
    p_a_and_b=count_city/df_denominator

    # count of specific  number in departure_city
    xx=df['fourth_city'].loc[lambda x: x==dep_city_code].index

    count_num_dc=0
    for i in range(len(xx)):
        count_num_dc+=df['count'][i]
        
    p_a=(count_num_dc/df_denominator)
    
    
    # count of specific  number in arrival_city
    xx=df['fourth_city'].loc[lambda x: x==arr_city_code].index

    count_num_ac=0
    for i in range(len(xx)):
        count_num_ac+=df['count'][i]

    p_b=(count_num_ac/df_denominator)
    
    #lift
    lift_a_b=p_a_and_b/((p_b)*(p_a))

    # p(a|b) = lift * p(a)
    p_a_given_b=lift_a_b*p_a
    
    return p_a_given_b

In [273]:
first_city=11213123

In [346]:
di=df['departure_city'].loc[lambda x: x==33408].index 
#mean, std of 
mu = np.mean(df['count'][di])
sigma = np.std(df['count'][di])
print(round(mu,2),round(sigma,2), round(mu+1.5*sigma,2))

4.78 11.97 22.73


In [276]:
def recommedation_1(first_city):
    if first_city  not in df['departure_city']: 
        return 0
    
    di=df['departure_city'].loc[lambda x: x==first_city].index 
    #mean, std of 
    mu = np.mean(df['count'][di])
    sigma = np.std(df['count'][di])
    
    # list of bigger than 1.5 sigma 
    index_15_sigma=df['count'][di].loc[lambda x: x>= (mu + 1.5*sigma)].index.values
    r1_list=(df['arrival_city'][index_15_sigma].values).tolist()
        
    # value of less than 1.5 sigma -> only return max value    
    max_index=(df['count'][di].loc[lambda x: x< (mu + 1.5*sigma)]).idxmax()
    max_index=np.array([max_index])
    r2_list=(df['arrival_city'][max_index].values).tolist()
        
    r_list=r1_list+r2_list    
    return r1_list

In [289]:
def final_four(first_city):
    
    #return second city list
    second_cities=recommedation_1(first_city)
    if second_cities ==[0]:
        return 0

    #return third city list 
    third_cities=[]
    for i in range(len(second_cities)):
        x=recommedation_1(second_cities[i])
        third_cities.append(x)
    third_cities=flatten(third_cities)
    
    
    if third_cities ==[0]:
        return 0

    #return fourth city list 
    fourth_cities=[]
    for i in range(len(third_cities)):
        x=recommedation_1(third_cities[i])
        fourth_cities.append(x)
    fourth_cities=flatten(fourth_cities)
    
        
    if fourth_cities ==[0]:
        return 0
    
    number_cities = collections.defaultdict(int)
    for elem in fourth_cities:
        number_cities[elem] += 1
    fourth_cities = pd.DataFrame(number_cities.items(), columns=['fourth_cities','count'])
    
    #insert lift value between fourth city and first city
    fourth_cities.insert(2, "p(a|b)", [lift_list(tff,first_city, fourth_cities['fourth_cities'][i]) for i in range(len(fourth_cities))], True)
    
    fourth_cities['count*p(a|b)']=fourth_cities['count']*fourth_cities['p(a|b)']
    
    #descening order by multiplied_lift
    fourth_cities=fourth_cities.sort_values('count*p(a|b)', ascending=False)
    
    final_list=fourth_cities.iloc[0:4]['fourth_cities'].tolist()
    
    return final_list

In [138]:
#return second city list
second_cities=recommedation_1(first_city)

#return third city list 
third_cities=[]
for i in range(len(second_cities)):
    x=recommedation_1(second_cities[i])
    third_cities.append(x)
third_cities=flatten(third_cities)

#return fourth city list 
fourth_cities=[]
for i in range(len(third_cities)):
    x=recommedation_1(third_cities[i])
    fourth_cities.append(x)
fourth_cities=flatten(fourth_cities)

In [171]:
len(fourth_cities)

114

In [352]:
#fourth_cities

In [121]:
# count  dupliate cities in fourth list 
number_cities = collections.defaultdict(int)
for elem in fourth_cities:
    number_cities[elem] += 1

In [175]:
#making dataframe
fourth_cities = pd.DataFrame(number_cities.items(), columns=['fourth_cities','count'])

In [353]:
#fourth_cities

In [354]:
#fourth_cities

In [178]:
#insert lift value between fourth city and first city
fourth_cities.insert(2, "p(a|b)", [lift_list(tff,first_city, fourth_cities['fourth_cities'][i]) for i in range(len(fourth_cities))], True)

In [180]:
fourth_cities['count*p(a|b)']=fourth_cities['count']*fourth_cities['p(a|b)']

In [182]:
#descening order by multiplied_lift
fourth_cities=fourth_cities.sort_values('count*p(a|b)', ascending=False)

In [355]:
#fourth_cities

In [184]:
final_list=fourth_cities.iloc[0:4]['fourth_cities'].tolist()

In [185]:
final_list

[49311, 11179, 47360, 65183]

In [186]:
testDataset=pd.read_csv('testDataset_first_fourth_city.csv')
testDataset

,first_city,fourth_city,count
0,56430,5797,1
1,50075,44768,1
2,47759,50457,14
3,8148,26235,2
4,46258,61619,1
...,...,...,...
42765,657,41237,1
42766,26575,61898,1
42767,58178,28073,1
42768,5429,21730,1


In [358]:
# Accuracy Test 

In [334]:
#test_k=testDataset.sample(1000)
test_k=testDataset[:100]
test_k = test_k.reset_index(drop=True)
test_k

,first_city,fourth_city,count
0,56430,5797,1
1,50075,44768,1
2,47759,50457,14
3,8148,26235,2
4,46258,61619,1
...,...,...,...
95,11473,17259,1
96,51339,62571,1
97,47486,45073,5
98,26103,40574,1


In [335]:
for i in range(len(test_k)):
    deno=len(test_k)
    
    first_city=test_k['first_city'][i]
    first_city=int(first_city)
    
    fourth_city=test_k['fourth_city'][i]
    fourth_city=int(fourth_city)

    acc=0
    #y=final_four(first_city)
    try:
        if fourth_city in final_four(first_city):
            k=int(test_k['count'][i])
            acc+=1*k
    except ValueError:
        continue

/Users/seungminyoo/opt/anaconda3/envs/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars


In [336]:
acc

30

In [338]:
acc/len(test_k)

0.3